In [80]:
import os
import pandas as pd
pd.set_option('precision', 2)

In [3]:
DATA_FOLDER = 'data'

# Load data

In [68]:
# https://www.bea.gov/news/2021/personal-income-county-and-metropolitan-area-2020
counties_df = pd.read_csv(
    os.path.join(DATA_FOLDER, 'county_income_2020.csv'),
    header=1,
)

# Force comma-delimited numbers to floats
for year in ['2018', '2019', '2020']:
    counties_df[year] = pd.to_numeric(counties_df[year].str.replace(',', ''), errors='coerce')

counties_df

,County,State,2018,2019,2020
0,United States,United States,54098.0,56047.0,59510.0
1,Alabama,Alabama,42328.0,43996.0,46479.0
2,Autauga,Alabama,42931.0,44368.0,46814.0
3,Baldwin,Alabama,46905.0,48270.0,50953.0
4,Barbour,Alabama,34064.0,35572.0,37850.0
...,...,...,...,...,...
3161,Sweetwater,Wyoming,54941.0,55947.0,54000.0
3162,Teton,Wyoming,217958.0,222893.0,220645.0
3163,Uinta,Wyoming,40205.0,41392.0,42854.0
3164,Washakie,Wyoming,48717.0,51138.0,54361.0


In [52]:
# https://www.bea.gov/news/2021/real-personal-consumption-expenditures-and-personal-income-state-2020
msa_rpp_df = pd.read_csv(os.path.join(DATA_FOLDER, 'msa_rpp_2020.csv'))

msa_rpp_df

,Metropolitan Statisical Area,All items,Goods,Housing,Utilities,Other
0,United States,100.0,99.7,100.4,98.2,100.2
1,Non-Metropolitan Areas,89.0,96.1,61.5,98.4,97.7
2,"Abilene, TX",94.4,97.0,80.0,111.6,99.8
3,"Akron, OH",93.0,93.7,78.8,95.5,97.3
4,"Albany, GA",84.6,95.0,49.6,109.6,95.3
...,...,...,...,...,...,...
381,"Yakima, WA",96.0,103.8,76.1,89.3,101.2
382,"York-Hanover, PA",97.2,98.5,87.6,95.2,99.5
383,"Youngstown-Warren-Boardman, OH-PA",87.6,93.8,58.7,95.5,97.3
384,"Yuba City, CA",99.5,103.2,88.5,95.3,101.4


In [20]:
# https://www.census.gov/geographies/reference-files/time-series/demo/metro-micro/delineation-files.html
# March 2020
delineation_df = pd.read_csv(os.path.join(DATA_FOLDER, 'delineation_file.csv'))

delineation_df

,CBSA Code,Metropolitan Division Code,CSA Code,CBSA Title,Metropolitan/Micropolitan Statistical Area,Metropolitan Division Title,CSA Title,County/County Equivalent,State Name,FIPS State Code,FIPS County Code,Central/Outlying County
0,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Brown County,South Dakota,46,13,Central
1,10100,NaN,NaN,"Aberdeen, SD",Micropolitan Statistical Area,NaN,NaN,Edmunds County,South Dakota,46,45,Outlying
2,10140,NaN,NaN,"Aberdeen, WA",Micropolitan Statistical Area,NaN,NaN,Grays Harbor County,Washington,53,27,Central
3,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Callahan County,Texas,48,59,Outlying
4,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,NaN,NaN,Jones County,Texas,48,253,Outlying
...,...,...,...,...,...,...,...,...,...,...,...,...
1911,49700,NaN,472.0,"Yuba City, CA",Metropolitan Statistical Area,NaN,"Sacramento-Roseville, CA",Sutter County,California,6,101,Central
1912,49700,NaN,472.0,"Yuba City, CA",Metropolitan Statistical Area,NaN,"Sacramento-Roseville, CA",Yuba County,California,6,115,Central
1913,49740,NaN,NaN,"Yuma, AZ",Metropolitan Statistical Area,NaN,NaN,Yuma County,Arizona,4,27,Central
1914,49780,NaN,198.0,"Zanesville, OH",Micropolitan Statistical Area,NaN,"Columbus-Marion-Zanesville, OH",Muskingum County,Ohio,39,119,Central


# Merge

In [53]:
# Only care about metropolitan areas
msa_only = (
    delineation_df['Metropolitan/Micropolitan Statistical Area'] == 'Metropolitan Statistical Area'
)

# Keep counties/states and CBSAs
county_to_cbsa_df = delineation_df[msa_only][['County/County Equivalent', 'State Name', 'CBSA Title']]

# Remove County from names
county_to_cbsa_df['County/County Equivalent'] = county_to_cbsa_df['County/County Equivalent'].str.replace(' County', '')
county_to_cbsa_df['County/County Equivalent'] = county_to_cbsa_df['County/County Equivalent'].str.replace(' Municipio', '')

county_to_cbsa_df.head()

,County/County Equivalent,State Name,CBSA Title
3,Callahan,Texas,"Abilene, TX"
4,Jones,Texas,"Abilene, TX"
5,Taylor,Texas,"Abilene, TX"
8,Aguada,Puerto Rico,"Aguadilla-Isabela, PR"
9,Aguadilla,Puerto Rico,"Aguadilla-Isabela, PR"


In [78]:
# Merge CBSA for each county
merge_df = pd.merge(
    counties_df,
    county_to_cbsa_df,
    how='left',
    left_on=['County', 'State'],
    right_on=['County/County Equivalent', 'State Name']
)

# Merge RPP for each CBSA
merge_df = pd.merge(
    merge_df,
    msa_rpp_df,
    how='left',
    left_on='CBSA Title',
    right_on='Metropolitan Statisical Area',
)

merge_df

,County,State,2018,2019,2020,County/County Equivalent,State Name,CBSA Title,Metropolitan Statisical Area,All items,Goods,Housing,Utilities,Other
0,United States,United States,54098.0,56047.0,59510.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Alabama,42328.0,43996.0,46479.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga,Alabama,42931.0,44368.0,46814.0,Autauga,Alabama,"Montgomery, AL","Montgomery, AL",90.6,95.1,64.9,118.5,97.8
3,Baldwin,Alabama,46905.0,48270.0,50953.0,Baldwin,Alabama,"Daphne-Fairhope-Foley, AL","Daphne-Fairhope-Foley, AL",96.9,95.1,95.5,118.5,97.8
4,Barbour,Alabama,34064.0,35572.0,37850.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3161,Sweetwater,Wyoming,54941.0,55947.0,54000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3162,Teton,Wyoming,217958.0,222893.0,220645.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3163,Uinta,Wyoming,40205.0,41392.0,42854.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,Washakie,Wyoming,48717.0,51138.0,54361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# Can only calculate for counties in a CBSA
msa_merge_df = merge_df[~merge_df['Metropolitan Statisical Area'].isna()].copy()

# Adjusting for All items RPP
msa_merge_df['2020 personal income (RPP All items)'] = msa_merge_df['2020']*(100/msa_merge_df['All items'])
msa_merge_df.sort_values(by='2020 personal income (RPP All items)', ascending=False, inplace=True)

msa_merge_df[['County', 'State', '2020 personal income (RPP All items)']].head(20)

,County,State,2020 personal income (RPP All items)
1892,New York,New York,165558.44
2467,Union,South Dakota,136666.67
214,Marin,California,123999.15
231,San Francisco,California,123354.34
2732,Midland,Texas,121294.06
234,San Mateo,California,120818.57
318,Fairfield,Connecticut,113867.42
2896,Goochland,Virginia,110711.74
236,Santa Clara,California,110313.11
342,Collier,Florida,102938.55
